# Deep Learning Toolkit for Splunk - Barebone Notebook

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import torch
import torch.utils.data
import torch.nn as nn
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.15.4
pandas version: 0.25.1


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone epochs=10 batch_size=1 s from feature_* into app:barebone_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'data'

In [4]:
# take a look at some matsnu example domains
from dga import matsnu

for i in range(10):
    print(matsnu.generate_domain())

teamreasonfallseekthinkglove.com
birddisappointedrideaddress.com
shoplunchguardgroundhighlight.com
teacherstrokesailnoteexplore.com
courageconsiderwordwishaccept.com
truthpurchaseaddressbirth.com
breathroofpagepotimagine.com
guyconsultclosetpersuade.com
meatbringbookloosediscover.com
cupbilllaysignmarketbedappreciate.com


In [5]:
# matsnu domains
matsnu_list = []

for i in range(20000):
    matsnu_list.append(matsnu.generate_domain())
    
matsnu_df = pd.DataFrame(matsnu_list, columns=['domain'])

print("Matsnu Shape:", matsnu_df.shape)

matsnu_df['label'] = 'dga'

matsnu_df.head()

Matsnu Shape: (20000, 1)


,domain,label
0,speakerstableservejoinvisit.com,dga
1,championshipwavedreamgrab.com,dga
2,angeroilrevealreflectreserve.com,dga
3,tripmarkwallweighgenerate.com,dga
4,bedroomcontactreasonappear.com,dga


In [6]:
# alex top 1 million domains
alexa_df = pd.read_csv(data_dir + "/alexa_top_1m.csv", names=['domain'])

alexa_df['label'] = 'benign'

alexa_df.head()

,domain,label
1,google.com,benign
2,youtube.com,benign
3,facebook.com,benign
4,baidu.com,benign
5,wikipedia.org,benign


In [7]:
train_df = pd.concat([alexa_df.iloc[:20000], matsnu_df.iloc[:20000]], axis=0).reset_index(drop=True)

In [8]:
train_df.head()

,domain,label
0,google.com,benign
1,youtube.com,benign
2,facebook.com,benign
3,baidu.com,benign
4,wikipedia.org,benign


In [9]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder

# instantiate labelencoder object
le = LabelEncoder()

In [10]:
# Apply le on categorical feature columns
train_df['label'] = le.fit_transform(train_df['label'])

train_df['label'].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

In [11]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from __future__ import unicode_literals, print_function, division
from io import open
import glob

# print(findFiles('data/names/*.txt'))

import unicodedata
import string

class DGADataset(Dataset):
    def __init__(self, data_dir, csv_path, n_records):
        """
        Args:
            data_dir (string): directory name
            csv_path (string): csv filename
        """
        
        self.alexa_df = pd.read_csv(
            os.path.join(data_dir, csv_path),
            names=['domain']
        ).iloc[:n_records]
        self.alexa_df['label'] = 'benign'
        
        self.matsnu_df = pd.DataFrame(
            [matsnu.generate_domain() for i in range(n_records)],
            columns = ['domain']
        )
        self.matsnu_df['label'] = 'dga'
        
        self.data_df = pd.concat([self.alexa_df, self.matsnu_df]).reset_index(drop=True)
        
        class_labeler = LabelEncoder()
        self.data_df['class'] = le.fit_transform(self.data_df['label'])
        
        self.inputs = self.data_df.iloc[:, 0]
        self.labels = self.data_df.iloc[:, -1]
        
        self.all_letters = string.ascii_letters + string.digits + " .'"
        self.n_letters = len(self.all_letters)
        
        self.data_len = len(self.data_df.index)
        
    # Find letter index from all_letters, e.g. "a" = 0
    def letterToIndex(self, letter):
        return self.all_letters.find(letter)

    # Just for demonstration, turn a letter into a <1 x n_letters> Tensor
    def letterToTensor(self, letter):
        tensor = torch.zeros(1, self.n_letters)
        tensor[0][self.letterToIndex(letter)] = 1
        return tensor

    # Turn a line into a <line_length x 1 x n_letters>,
    # or an array of one-hot letter vectors
    def lineToTensor(self, line):
        tensor = torch.zeros(len(line), 1, self.n_letters)
        for li, letter in enumerate(line):
            tensor[li][0][self.letterToIndex(letter)] = 1
        return tensor

    def __getitem__(self, index):
        domain = self.lineToTensor(self.inputs[index]).float()
        target = torch.tensor([self.labels[index]]).float()
        return (domain, target)

    def __len__(self):
        return self.data_len


In [12]:
DGADataset('data', 'alexa_top_1m.csv', 10).__getitem__(0)

(tensor([[[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [88]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# data_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, collate_fn=pad_collate)

def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)

    return xx_pad, yy, x_lens, y_lens

def _get_train_data_loader(batch_size, data_dir, filename, n_records):
    print("Get train data loader.")
    
    dataset =  DGADataset(data_dir, filename, n_records)
    
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)

In [89]:
import unittest

class TestDGADataSet(unittest.TestCase):
    def setUp(self):
        self.n = 200
        self.test_dga = DGADataset('data', 'alexa_top_1m.csv', self.n)

    def test_length(self):
        actual_len = self.test_dga.__len__()
        self.assertEqual(actual_len, self.n * 2, 
                         "DataSet is not the right length ({}), should be {}".format(actual_len, 400)
                        )

    def test_labels(self):
        expected_labels = ['benign', 'dga']
        labels = self.test_dga.data_df['label'].unique().tolist()
        self.assertEqual(labels, 
                         expected_labels, 
                         "Incorrect labels ({}), expected {}".format(labels, expected_labels)
                        )
        
    def test_classes(self):
        expected_classes = [0, 1]
        classes = self.test_dga.data_df['class'].unique().tolist()
        self.assertEqual(classes, 
                         expected_classes, 
                         "Incorrect classes ({}), expected {})".format(classes, expected_classes)
                        )
        
    def test_index(self):
        index = self.test_dga.data_df.index.tolist()
        self.assertEqual(len(index), 
                         len(set(index)),
                         "Index contains duplicates ({}), expected {})".format(len(set(index)), len(index)))

    def test_getitem(self):
        item = self.test_dga.__getitem__(0)
        self.assertEqual(len(item), 
                         2,
                         "Get item output is wrong length ({}), expected {})".format(len(item), 2))
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 4.274s

OK


In [90]:
# from __future__ import unicode_literals, print_function, division
# from io import open
# import glob
# import os

# # print(findFiles('data/names/*.txt'))

# import unicodedata
# import string

# all_letters = string.ascii_letters + string.digits + " .'"
# n_letters = len(all_letters)

In [91]:
# # Find letter index from all_letters, e.g. "a" = 0
# def letterToIndex(letter):
#     return all_letters.find(letter)

# # Just for demonstration, turn a letter into a <1 x n_letters> Tensor
# def letterToTensor(letter):
#     tensor = torch.zeros(1, n_letters)
#     tensor[0][letterToIndex(letter)] = 1
#     return tensor

# # Turn a line into a <line_length x 1 x n_letters>,
# # or an array of one-hot letter vectors
# def lineToTensor(line):
#     tensor = torch.zeros(len(line), 1, n_letters)
#     for li, letter in enumerate(line):
#         tensor[li][0][letterToIndex(letter)] = 1
#     return tensor

# print(letterToTensor('b'))

# print(lineToTensor('blindpagegolucksinkorganize.com').size())

# print(len('blindpagegolucksinkorganize.com'))

In [92]:

# for i, row in train_df.iterrows():
#     print(i, row)
#     lineToTensor('blindpagegolucksinkorganize.com')
    
#     break

In [93]:
# import random

# def randomTrainingExample():
#     category = random.randint(0, 1)
#     line = train_df[train_df['label'] == category].sample()['domain'].values[0]
#     category_tensor = torch.tensor([category], dtype=torch.long)
#     line_tensor = lineToTensor(line)
#     return category, line, category_tensor, line_tensor

# for i in range(10):
#     category, line, category_tensor, line_tensor = randomTrainingExample()
#     print('category =', category, '/ line =', line)

In [94]:
# TO:DO Figure out splunk query
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
# def stage(name):
#     with open("data/"+name+".csv", 'r') as f:
#         df = pd.read_csv(f)
#     with open("data/"+name+".json", 'r') as f:
#         param = json.load(f)
#     return df, param

batch_size = 10
data_dir = 'data'
filename = 'alexa_top_1m.csv'
n_records = 100

In [95]:
# # THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df = _get_train_data_loader(batch_size, data_dir, filename, n_records)# print(df.describe())
print(df)
param = {}
print(param)

Get train data loader.
{}


In [105]:
# for i, j in enumerate(df):
#     print(i, j[1].long)
#     break

SyntaxError: invalid syntax (<ipython-input-105-47c156b4cbd5>, line 2)

In [225]:
from torch.nn.utils.rnn import pack_padded_sequence


class RNNEstimator(nn.Module):
    """
    RNN Estimator for generating sequences of target variables.
    """
    
    def __init__(self, input_features=65, hidden_dim=12, n_layers=2, output_dim=9, embedding_dim=5, batch_size=10):
        super(RNNEstimator, self).__init__()

        self.hidden_dim = hidden_dim
        self.hidden_layers = n_layers
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        
        # Embedding
        self.embedding = nn.Embedding(input_features, embedding_dim)
        
        # RNN Layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, dropout=0.3, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.softmax = nn.LogSoftmax(dim=1)
        
    def init_hidden(self):
        """
        Initialize the hidden and cell states of the LSTM with zeros.
        """
        return torch.zeros(self.hidden_layers, self.batch_size, self.embedding_dim)
        
    def forward(self, x, x_lens, hidden_state):
        """
        Perform a forward pass of our model on batch of tracks.
        """
        print(x.shape)
        # x: (batch_size, longest_sequence, 1?, all_chars/inputs) i.e. 10, 32, 1, 65
        embed_x = self.embedding(x.squeeze(2).long())
        print(embed_x.shape)
        x_packed = pack_padded_sequence(embed_x, x_lens, batch_first=True, enforce_sorted=False)
        
        # Passing in the input and hidden state into the model and obtaining outputs
        output_packed, hidden_state = self.rnn(x_packed, hidden_state)
        
        output_padded, output_lengths = pad_packed_sequence(output_packed, batch_first=True)
        
        output = self.softmax(self.fc(output_padded))
        
        return output, hidden_state

## Stage 2 - create and initialize a model

In [226]:
def init(df, param):
    
#     X = df[param['feature_variables']]
#     Y = df[param['target_variables']]
    
    input_size = 65
    hidden_dim = 30
    num_classes = 2
    learning_rate = 0.001
    embedding_dim = 5
    n_layers = 2
    
#     mapping = { key: value for value,key in enumerate(np.unique(Y.to_numpy().reshape(-1))) }
    
#     # TODO: Change these
#     print("FIT build RNN model with input shape " + str(X.shape))
#     print("FIT build model with target classes " + str(num_classes))
    
    model = {
        "input_size": input_size,
        "hidden_dim": hidden_dim,
        "embedding_dim": embedding_dim,
        "num_classes": num_classes,
        "n_layers": n_layers,
        "learning_rate": learning_rate,
#         "mapping": mapping,
        "num_epochs": 500,
        "batch_size": 10,
    }
    
    model["train_loader"] = df
    
#     if 'options' in param:
#         if 'params' in param['options']:
#             if 'epochs' in param['options']['params']:
#                 model['num_epochs'] = int(param['options']['params']['epochs'])
#             if 'batch_size' in param['options']['params']:
#                 model['batch_size'] = int(param['options']['params']['batch_size'])

    # RNN Layer
    model['model'] = RNNEstimator(
        input_features=model["input_size"], 
        hidden_dim=model["hidden_dim"], 
        n_layers=model["n_layers"], 
        output_dim=model["num_classes"],
        embedding_dim=model["embedding_dim"], 
        batch_size=model["batch_size"]
    )
    
    # Define loss and optimizer
    model['criterion'] = torch.nn.NLLLoss()
    
    model['optimizer'] = torch.optim.SGD(model['model'].parameters(), lr=model["learning_rate"])
    
    return model

In [227]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'input_size': 65, 'hidden_dim': 30, 'embedding_dim': 5, 'num_classes': 2, 'n_layers': 2, 'learning_rate': 0.001, 'num_epochs': 500, 'batch_size': 10, 'train_loader': <torch.utils.data.dataloader.DataLoader object at 0x7f2e37f3ce80>, 'model': RNNEstimator(
  (embedding): Embedding(65, 5)
  (rnn): RNN(5, 30, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=30, out_features=2, bias=True)
  (softmax): LogSoftmax()
), 'criterion': NLLLoss(), 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)}


In [228]:
# def make_target(label, label_to_ix):
#     return torch.LongTensor([label_to_ix[label]])

# label_to_ix = {"benign": 0, "dga": 1}

In [229]:
# all_categories = ["benign", "dga"]
# def categoryFromOutput(output):
#     top_n, top_i = output.topk(1)
#     category_i = top_i[0].item()
#     return all_categories[category_i], category_i

## Stage 3 - fit the model

In [230]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model, df, param):
    # model.fit()
    
    returns = {}
    info = {"message": "model trained"}
    
    model['model'].train() # Make sure that the model is in training mode.

    for epoch in range(1, model['num_epochs'] + 1):

        avg_loss = 0

        # Iterate over dataset
        for _, (x_padded, y_padded, x_lens, y_lens) in enumerate(model["train_loader"]):
            
            # Clear stored gradient
            model["optimizer"].zero_grad()

            # Initialize hidden state
            hidden_cell =  model['model'].init_hidden()

            # Forward pass
            output, hidden_cell =  model['model'](x_padded, x_lens, hidden_cell)
            
            # Calculate MAE loss over batch
            batch_loss =  model['criterion'](output, batch_y.long())
            avg_loss += batch_loss.item()

            # Zero out gradient, so it doesnt accumulate between epochs
            model["optimizer"].zero_grad()

            # Backward pass
            batch_loss.backward()

            # Update parameters
            model["optimizer"].step()
            
#             if epoch % 50 == 0:
#                 guess, guess_i = categoryFromOutput(output)
#                 correct = '✓' if guess_i == category else '✗ (%s)' % category
#                 print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

        if epoch % 50 == 0:
            print('Epoch: {}/{}.............'.format(epoch, model['num_epochs']), end=' ')
            print("Loss: {:.4f}".format(avg_loss / len(model["train_loader"])))
            
    # memorize parameters
    returns['model_epochs'] = model['num_epochs']
    returns['model_batch_size'] = model['batch_size']
    returns['model_loss_acc'] = loss.item()
            
    return info

In [231]:
# !pip install --upgrade torch torchvision

In [232]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

torch.Size([10, 35, 1, 65])
torch.Size([10, 35, 65, 5])


RuntimeError: input must have 2 dimensions, got 3

## Stage 4 - apply the model

In [168]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    y_hat = np.ones(df.shape[0]) * model['hyperparameter']
    result = pd.DataFrame(y_hat, columns=['meaning_of_life'])
    return result

In [116]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

AttributeError: 'DataLoader' object has no attribute 'shape'

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code